In [1]:
import requests
import json
import pandas as pd
import time
import random

In [2]:
from datetime import datetime, timedelta

# Current time in UTC
now = datetime.utcnow()

# Calculate the start time and end time
start_time = now.replace(minute=0, second=0, microsecond=0)
start_time = start_time.strftime("%Y-%m-%dT%H:%M:%SZ")
end_time = (now + timedelta(weeks=1)).strftime("%Y-%m-%dT%H:%M:%SZ")  # 2 weeks later
end_time = (now + timedelta(weeks=2)).replace(
        hour=23,
        minute=59,
        second=59,
        microsecond=0
    )
end_time = end_time.strftime("%Y-%m-%dT%H:%M:%SZ")

# Construct the URL
print(start_time)
print(end_time)

2025-02-08T13:00:00Z
2025-02-22T23:59:59Z


In [3]:
type(end_time)

str

In [ ]:
import requests
import time
import random
import pandas as pd

# Initialize an empty list to store each batch of matches data
all_matches = []

# Loop over drillDownTagIds ranges in steps of 900
for start in range(200, 14000, 200):
    # Generate a comma-separated string of drillDownTagIds for the current range
    drilldown_ids = ','.join(str(i) for i in range(start, start + 200))

    # Define the request URL
    url = (
        f"https://content.toto.nl/content-service/api/v1/q/event-list?"
        f"startTimeFrom={start_time}"
        f"startTimeTo={end_time}&"
        "liveNow=false&"
        "maxEvents=190&"
        "orderEventsBy=popularity&"
        "orderMarketsBy=displayOrder&"
        "marketSortsIncluded=--,CS,DC,DN,HH,HL,MH,MR,WH&"
        "marketGroupTypesIncluded=CUSTOM_GROUP,DOUBLE_CHANCE,DRAW_NO_BET,MATCH_RESULT,"
        "MATCH_WINNER,MONEYLINE,ROLLING_SPREAD,ROLLING_TOTAL,STATIC_SPREAD,STATIC_TOTAL&"
        "eventSortsIncluded=MTCH&"
        "includeChildMarkets=true&"
        "prioritisePrimaryMarkets=true&"
        "includeCommentary=true&"
        "includeMedia=true&"
        f"drilldownTagIds={drilldown_ids}&"
        "lang=nl-NL&"
        "channel=I"
    )

    # Headers for the request
    headers = {
        'accept': 'application/json',
        'accept-language': 'en-US,en;q=0.9,nl;q=0.8',
    }

    try:
        # Make the request
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise HTTPError for bad responses

        # Parse the JSON response
        data = response.json()

        # Extract match data if available
        matches = []
        for event in data.get('data', {}).get('events', []):
            match = {
                "event_id": event.get("id"),
                "match_name": event.get("name"),
                "start_time": event.get("startTime"),
                "home_team": next((team['name'] for team in event.get('teams', []) if team['side'] == "HOME"), None),
                "away_team": next((team['name'] for team in event.get('teams', []) if team['side'] == "AWAY"), None),
                "competition": event.get('type', {}).get('name'),
                "country": event.get('class', {}).get('name'),
                "sport": event.get('category', {}).get('name'),
            }

            # Extract odds if available
            outcomes = event.get("markets", [{}])[0].get("outcomes", [])
            match["home_odds"] = next((outcome['prices'][0]['decimal'] for outcome in outcomes if outcome.get('subType') == "H"), None)
            match["away_odds"] = next((outcome['prices'][0]['decimal'] for outcome in outcomes if outcome.get('subType') == "A"), None)
            
            matches.append(match)

        # Add the batch of matches to the all_matches list
        all_matches.extend(matches)

        # # Sleep for a random duration between 1 and 4 seconds, with variability of 0.1 ms
        # time.sleep(random.uniform(0, 1.001))

    except Exception as e:
        print(f"An error occurred for drilldown ID range {start}–{start + 200}: {e}")

# Combine all match data into a single DataFrame
final_matches_df = pd.DataFrame(all_matches)

# Display the resulting DataFrame
print(final_matches_df)

911
   event_id                               match_name            start_time  \
0   6692298                   LOSC Lille vs Le Havre  2025-02-08T18:00:00Z   
1   6692300        Angers SCO vs Olympique Marseille  2025-02-09T19:45:00Z   
2   6692259                      OGC Nice vs RC Lens  2025-02-08T16:00:00Z   
3   6555223            Olympique Lyon vs Stade Reims  2025-02-09T14:00:00Z   
4   6612005         AS Saint-Etienne vs Stade Rennes  2025-02-08T20:05:00Z   
5   6692296                Aj Auxerre vs Toulouse FC  2025-02-09T16:15:00Z   
6   6692297            Strasbourg vs Montpellier Hsc  2025-02-09T16:15:00Z   
7   6605962                      AS Monaco vs Nantes  2025-02-15T18:00:00Z   
8   6761243                     Le Havre vs OGC Nice  2025-02-16T16:15:00Z   
9   6761263        Montpellier Hsc vs Olympique Lyon  2025-02-16T14:00:00Z   
10  6761214  Olympique Marseille vs AS Saint-Etienne  2025-02-15T16:00:00Z   
11  6761216                    RC Lens vs Strasbourg  2025-0

In [23]:
import requests
import time
import random
import pandas as pd
from itertools import islice

# Define the country drilldown IDs
COUNTRY_DRILLDOWNS = {
    'Nederland': 1052,
    'Belgie': 864,
    'Engeland': 40,
    'Spanje': 44,
    'Italie': 43,
    'Duitsland': 42,
    'Frankrijk': 41,
    'Portugal': 551,
    'Turkije': 537,
    'Saudie Arabie': 987,
    'VS': 595,
    'Europees': 3214,
    'Wereldwijd': 667,
    'Americas': 9342,
    'Africa': 6505,
    'Andorra': 4578,
    'Algerije': 1797,
    'Argentinie': 1670,
    'Australie': 825,
    'Azerbeidzjan': 786,
    'Bahrein': 1000,
    'Bosnia Herzegovina': 963,
    'Brazilie': 896,
    'Bulgarije': 619,
    'Burundi': 519,
    'Chili': 1132,
    'Cambodja': 4125,
    'Costa Rica': 475,
    'Colombia': 1225
}

def batch_dict(data, batch_size):
    it = iter(data)
    for i in range(0, len(data), batch_size):
        batch = {k: data[k] for k in islice(it, batch_size)}
        yield batch

# Initialize an empty list to store each batch of matches data
all_matches = []

# Process countries in batches of 4
for batch_num, country_batch in enumerate(batch_dict(COUNTRY_DRILLDOWNS, 1)):
    print(f"Processing batch {batch_num + 1} with countries: {', '.join(country_batch.keys())}")
    
    # Convert batch dictionary values to comma-separated string
    drilldown_ids = ','.join(str(id) for id in country_batch.values())

    # Define the request URL
    url = (
        f"https://content.toto.nl/content-service/api/v1/q/event-list?"
        f"startTimeFrom={start_time}"
        f"startTimeTo={end_time}&"
        "liveNow=false&"
        "maxEvents=190&"
        "orderEventsBy=popularity&"
        "orderMarketsBy=displayOrder&"
        "marketSortsIncluded=--,CS,DC,DN,HH,HL,MH,MR,WH&"
        "marketGroupTypesIncluded=CUSTOM_GROUP,DOUBLE_CHANCE,DRAW_NO_BET,MATCH_RESULT,"
        "MATCH_WINNER,MONEYLINE,ROLLING_SPREAD,ROLLING_TOTAL,STATIC_SPREAD,STATIC_TOTAL&"
        "eventSortsIncluded=MTCH&"
        "includeChildMarkets=true&"
        "prioritisePrimaryMarkets=true&"
        "includeCommentary=true&"
        "includeMedia=true&"
        f"drilldownTagIds={drilldown_ids}&"
        "lang=nl-NL&"
        "channel=I"
    )

    # Headers for the request
    headers = {
        'accept': 'application/json',
        'accept-language': 'en-US,en;q=0.9,nl;q=0.8',
    }

    try:
        # Make the request
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise HTTPError for bad responses

        # Parse the JSON response
        data = response.json()

        # Extract match data if available
        matches = []
        for event in data.get('data', {}).get('events', []):
            match = {
                "event_id": event.get("id"),
                "match_name": event.get("name"),
                "start_time": event.get("startTime"),
                "home_team": next((team['name'] for team in event.get('teams', []) if team['side'] == "HOME"), None),
                "away_team": next((team['name'] for team in event.get('teams', []) if team['side'] == "AWAY"), None),
                "competition": event.get('type', {}).get('name'),
                "country": event.get('class', {}).get('name'),
                "sport": event.get('category', {}).get('name'),
            }

            # Extract odds if available
            outcomes = event.get("markets", [{}])[0].get("outcomes", [])
            match["home_odds"] = next((outcome['prices'][0]['decimal'] for outcome in outcomes if outcome.get('subType') == "H"), None)
            match["away_odds"] = next((outcome['prices'][0]['decimal'] for outcome in outcomes if outcome.get('subType') == "A"), None)
            
            matches.append(match)

        # Add the batch of matches to the all_matches list
        all_matches.extend(matches)

        # Add a small delay between batches to be respectful to the API
        if batch_num < len(COUNTRY_DRILLDOWNS) // 1:  # Don't sleep after the last batch
            time.sleep(random.uniform(1, 2))

    except Exception as e:
        print(f"An error occurred while fetching batch {batch_num + 1}: {e}")
        continue  # Continue with the next batch even if this one fails

# Create the final DataFrame
final_matches_df = pd.DataFrame(all_matches)

# Display the resulting DataFrame
print(final_matches_df)

Processing batch 1 with countries: Nederland
Processing batch 2 with countries: Belgie
Processing batch 3 with countries: Engeland
Processing batch 4 with countries: Spanje
Processing batch 5 with countries: Italie
Processing batch 6 with countries: Duitsland
Processing batch 7 with countries: Frankrijk


KeyboardInterrupt: 

In [9]:
final_matches_df[final_matches_df['match_name'].str.contains('Girona')]

,event_id,match_name,start_time,home_team,away_team,competition,country,sport,home_odds,away_odds
57,6019489,Basquet Girona vs Basket Zaragoza,2025-02-09T16:00:00Z,Basquet Girona,Basket Zaragoza,Spanje ACB,Spanje,Basketbal,1.91,1.80
1741,6815501,Girona vs Madrid,2025-02-08T13:16:00Z,Girona,Madrid,La Liga 23/24,Virtual,Virtual Voetbal,3.15,2.15
1991,6815501,Girona vs Madrid,2025-02-08T13:16:00Z,Girona,Madrid,La Liga 23/24,Virtual,Virtual Voetbal,3.15,2.15


In [5]:
final_matches_df['event_id'].nunique()

1418

In [6]:
final_matches_df[final_matches_df['match_name'].str.contains('PSV')]

,event_id,match_name,start_time,home_team,away_team,competition,country,sport,home_odds,away_odds
505,6351955,NEC vs PSV,2025-02-01T20:00:00Z,NEC,PSV,Nederland Eredivisie,Nederland,Voetbal,7.00,1.36
513,6626894,Jong PSV vs VVV Venlo,2025-01-27T19:00:00Z,Jong PSV,VVV Venlo,Nederland Eerste Divisie,Nederland,Voetbal,2.40,2.65
520,6682208,PSV Eindhoven vs FC Twente,2025-02-01T15:30:00Z,PSV Eindhoven,FC Twente,Nederland Eredivisie Vrouwen,Nederland,Voetbal,1.36,5.75
1153,6248696,PSV vs Liverpool,2025-01-29T20:00:00Z,PSV,Liverpool,UEFA Champions League,Europees,Voetbal,2.80,2.37


In [7]:
import requests
import time
import random
import pandas as pd

# Initialize an empty list to accumulate all data
all_data = []

# Set batch size
batch_size = 30

# Loop over event_ids in batches of 500
for i in range(0, len(final_matches_df['event_id']), batch_size):
    try:
        # Get a batch of event_ids
        batch_ids = final_matches_df['event_id'][i:i + batch_size]
        
        # Concatenate event_ids for the URL
        event_ids_str = ','.join(map(str, batch_ids))
        
        # Define the URL for the current batch
        url = (
            f"https://content.toto.nl/content-service/api/v1/q/events-by-ids?"
            f"eventIds={event_ids_str}&"
            "includeChildMarkets=true&includeCollections=true&includePriorityCollectionChildMarkets=true&"
            "includePriceHistory=false&includeCommentary=true&includeIncidents=false&includeRace=false&"
            "includePools=false&includeNonFixedOdds=false&lang=nl-NL&channel=I"
        )
        
        headers = {
            'accept': 'application/json',
            'accept-language': 'en-US,en;q=0.9,nl;q=0.8',
            'Cookie': 'incap_ses_1581_2280942=FvpNPI919nl1UsyZNNfwFR6GL2cAAAAA8yX5UHfYkaWD/ax05SY+Jg==; visid_incap_2280942=CzAo1Y6yTMa654jo8RfVRtBiL2cAAAAAQkIPAAAAAAAE7rpFtkP20364qTw5VQQQ'
        }
        
        # Send request
        response = requests.get(url, headers=headers)
        json_data = response.json()

        # Initialize lists to store the data for this batch
        event_ids = []
        event_names = []
        market_names = []
        outcome_names = []
        odds_decimal = []
        prices_numerator = []
        prices_denominator = []
        outcome_types = []
        outcome_subtypes = []

        # Process the JSON response for each event
        for event in json_data['data']['events']:  # Iterate over each event
            event_id = event['id']
            event_name = event['name']
            
            for market in event['markets']:  # Iterate over each market in the event
                market_name = market['name']
                
                for outcome in market['outcomes']:  # Iterate over outcomes for each market
                    outcome_name = outcome['name']
                    odds_decimal_value = outcome['prices'][0]['decimal']
                    numerator = outcome['prices'][0]['numerator']
                    denominator = outcome['prices'][0]['denominator']
                    outcome_type = outcome['type']
                    outcome_subtype = outcome['subType']
                    
                    # Append captured values to the respective lists
                    event_ids.append(event_id)
                    event_names.append(event_name)
                    market_names.append(market_name)
                    outcome_names.append(outcome_name)
                    odds_decimal.append(odds_decimal_value)
                    prices_numerator.append(numerator)
                    prices_denominator.append(denominator)
                    outcome_types.append(outcome_type)
                    outcome_subtypes.append(outcome_subtype)

        # Create a temporary DataFrame for this batch and append it to all_data
        temp_df = pd.DataFrame({
            'event_id': event_ids,
            'Event Name': event_names,
            'Market Name': market_names,
            'Outcome Name': outcome_names,
            'Odds (Decimal)': odds_decimal,
            'Price Numerator': prices_numerator,
            'Price Denominator': prices_denominator,
            'Outcome Type': outcome_types,
            'Outcome SubType': outcome_subtypes
        })
        
        all_data.append(temp_df)
        
        # Wait between .. and .. seconds
        time.sleep(random.uniform(0.02, 1.81))
    except:
        None

# Concatenate all individual DataFrames into a final DataFrame
final_df = pd.concat(all_data, ignore_index=True)

# Show the final DataFrame
print(final_df)

       event_id                                       Event Name  \
0       6588685            TSG Hoffenheim vs Eintracht Frankfurt   
1       6588685            TSG Hoffenheim vs Eintracht Frankfurt   
2       6588685            TSG Hoffenheim vs Eintracht Frankfurt   
3       6588685            TSG Hoffenheim vs Eintracht Frankfurt   
4       6588685            TSG Hoffenheim vs Eintracht Frankfurt   
...         ...                                              ...   
115813  6674995  Pearson K / Sheehy J vs Puttergill C / Sweeny D   
115814  6674995  Pearson K / Sheehy J vs Puttergill C / Sweeny D   
115815  6674995  Pearson K / Sheehy J vs Puttergill C / Sweeny D   
115816  6674995  Pearson K / Sheehy J vs Puttergill C / Sweeny D   
115817  6674995  Pearson K / Sheehy J vs Puttergill C / Sweeny D   

                Market Name                 Outcome Name  Odds (Decimal)  \
0               Draw No Bet               TSG Hoffenheim            2.41   
1               Draw No Bet    

In [8]:
final_df[final_df['Event Name'].str.contains('PSV')]['Event Name'].unique()

array(['NEC vs PSV', 'Jong PSV vs VVV Venlo',
       'PSV Eindhoven vs FC Twente', 'PSV vs Liverpool'], dtype=object)

In [9]:
# Perform the left join on 'event_id' to get 'sport' and 'group_name' from events_df
final_df = final_df.merge(final_matches_df[['event_id', 'sport', 'competition']], on='event_id', how='left').drop_duplicates()
# Remove columns that end with '_x'
final_df = final_df[[col for col in final_df.columns if not col.endswith('_x')]]

# Rename columns to remove '_y'
final_df.columns = final_df.columns.str.replace('_y', '', regex=False)

In [10]:
# Replace 'A' with 2 and 'H' with 1 in the outcome_subtype column
final_df['Outcome SubType'] = final_df['Outcome SubType'].replace({'A': '2', 'H': '1'})

# Display the updated final_df
print(final_df)

       event_id                                       Event Name  \
0       6588685            TSG Hoffenheim vs Eintracht Frankfurt   
1       6588685            TSG Hoffenheim vs Eintracht Frankfurt   
2       6588685            TSG Hoffenheim vs Eintracht Frankfurt   
3       6588685            TSG Hoffenheim vs Eintracht Frankfurt   
4       6588685            TSG Hoffenheim vs Eintracht Frankfurt   
...         ...                                              ...   
175067  6674995  Pearson K / Sheehy J vs Puttergill C / Sweeny D   
175068  6674995  Pearson K / Sheehy J vs Puttergill C / Sweeny D   
175069  6674995  Pearson K / Sheehy J vs Puttergill C / Sweeny D   
175070  6674995  Pearson K / Sheehy J vs Puttergill C / Sweeny D   
175071  6674995  Pearson K / Sheehy J vs Puttergill C / Sweeny D   

                Market Name                 Outcome Name  Odds (Decimal)  \
0               Draw No Bet               TSG Hoffenheim            2.41   
1               Draw No Bet    

In [11]:
set(final_df[final_df['sport'].str.contains('Voetbal')].competition)

{'Andorra First Division',
 'Argentinië Superliga',
 'Australië A-League',
 'Australië W-League',
 'Azerbeidzjan Premier League',
 'België 1B Pro League',
 'België Jupiler Pro League',
 'Brazil Campeonato Paranaense',
 'Brazil Gaucho A1',
 'Brazil Supercup',
 'Brazilië Cearense',
 'Brazilië Mineiro Modulo I',
 'Brazilië Paulista A1\n',
 'Colombia Categoría Primera A',
 'Costa Rica Ascenso League',
 'Costa Rica Primera Division',
 'Cyprus Beker',
 'Cyprus First Division',
 'Duitsland 2.Bundesliga',
 'Duitsland 3.Liga',
 'Duitsland Bundesliga',
 'Egypte Premier League',
 'Egypte Second Division',
 'El Salvador Primera Division',
 'Engeland Championship',
 'Engeland FA Super League Vrouwen',
 'Engeland League 1',
 'Engeland League 2',
 'Engeland National League',
 'Engeland National League Noord',
 'Engeland National League Zuid',
 'Engeland Premier League',
 'Ethiopië Premier League',
 'Europa Super League',
 'Europees Kampioenschap',
 'Frankrijk Ligue 1',
 'Frankrijk Ligue 2',
 'Frankri

In [12]:
final_df[(final_df['Event Name'].str.contains('Milan')) & final_df['sport']=='Voetbal']

,event_id,Event Name,Market Name,Outcome Name,Odds (Decimal),Price Numerator,Price Denominator,Outcome Type,Outcome SubType,sport,competition


In [13]:
[i for i in list(set(final_df['Market Name'])) if 'No Bet' in i]

['1e Helft: Draw No Bet',
 '3e Periode - Draw No Bet',
 '2e Helft: Draw No Bet',
 '2e Periode - Draw No Bet',
 'Draw No Bet',
 '1e Periode - Draw No Bet']

In [14]:
final_output_name = 'totoAllSports' + start_time + '.csv'
final_df.to_csv(final_output_name)

## Check id ranges

In [106]:
import requests
import time
import random
import pandas as pd

# Initialize an empty list to store all match data
all_matches = []

# Initialize a dictionary to store sports and competitions by drillDownTagIds range
range_summary = {}

# Loop over drillDownTagIds ranges in steps of 1000
for start in range(0, 14000, 20):
    # Generate a comma-separated string of drillDownTagIds for the current range
    drilldown_ids = ','.join(str(i) for i in range(start, start + 200))

    # Define the request URL
    url = (
        f"https://content.toto.nl/content-service/api/v1/q/event-list?"
        "startTimeFrom=2025-01-30T21:00:00Z&"
        "startTimeTo=2025-02-30T22:59:59Z&"
        "liveNow=false&"
        "maxEvents=190&"
        "orderEventsBy=popularity&"
        "orderMarketsBy=displayOrder&"
        "marketSortsIncluded=--,CS,DC,DN,HH,HL,MH,MR,WH&"
        "marketGroupTypesIncluded=CUSTOM_GROUP,DOUBLE_CHANCE,DRAW_NO_BET,MATCH_RESULT,"
        "MATCH_WINNER,MONEYLINE,ROLLING_SPREAD,ROLLING_TOTAL,STATIC_SPREAD,STATIC_TOTAL&"
        "eventSortsIncluded=MTCH&"
        "includeChildMarkets=true&"
        "prioritisePrimaryMarkets=true&"
        "includeCommentary=true&"
        "includeMedia=true&"
        f"drilldownTagIds={drilldown_ids}&"
        "lang=nl-NL&"
        "channel=I"
    )

    # Headers for the request
    headers = {
        'accept': 'application/json',
        'accept-language': 'en-US,en;q=0.9,nl;q=0.8',
    }

    try:
        # Make the request
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise HTTPError for bad responses

        # Parse the JSON response
        data = response.json()

        # Extract match data if available
        matches = []
        sports_set = set()
        competitions_set = set()

        for event in data.get('data', {}).get('events', []):
            # Collect match data
            match = {
                "event_id": event.get("id"),
                "match_name": event.get("name"),
                "start_time": event.get("startTime"),
                "home_team": next((team['name'] for team in event.get('teams', []) if team['side'] == "HOME"), None),
                "away_team": next((team['name'] for team in event.get('teams', []) if team['side'] == "AWAY"), None),
                "competition": event.get('type', {}).get('name'),
                "country": event.get('class', {}).get('name'),
                "sport": event.get('category', {}).get('name'),
            }

            # Collect sports and competitions
            sports_set.add(match["sport"])
            competitions_set.add(match["competition"])

            # Extract odds if available
            outcomes = event.get("markets", [{}])[0].get("outcomes", [])
            match["home_odds"] = next((outcome['prices'][0]['decimal'] for outcome in outcomes if outcome.get('subType') == "H"), None)
            match["away_odds"] = next((outcome['prices'][0]['decimal'] for outcome in outcomes if outcome.get('subType') == "A"), None)

            matches.append(match)

        # Add matches to the all_matches list
        all_matches.extend(matches)

        # Add sports and competitions to the range summary
        range_summary[f"{start}-{start + 20}"] = {
            "sports": list(sports_set),
            "competitions": list(competitions_set)
        }

        # Sleep for a random duration between 1 and 2 seconds
        # time.sleep(random.uniform(1, 2.001))

    except Exception as e:
        print(f"An error occurred for drilldown ID range {start}–{start + 20}: {e}")

# Combine all match data into a single DataFrame
final_matches_df = pd.DataFrame(all_matches)

# Display the range summary
for range_key, summary in range_summary.items():
    print(f"\nDrillDownTagIds Range: {range_key}")
    print(f"Sports: {summary['sports']}")
    print(f"Competitions: {summary['competitions']}")

# Save the match data to a CSV or Excel file if needed
final_matches_df.to_csv("matches_output.csv", index=False)

# Optionally, save the range summary to a file
pd.DataFrame.from_dict(range_summary, orient="index").to_csv(f"range_summary_{datetime.utcnow()}.csv")

An error occurred for drilldown ID range 0–20: 'NoneType' object is not iterable
An error occurred for drilldown ID range 500–520: 'NoneType' object is not iterable
An error occurred for drilldown ID range 980–1000: 'NoneType' object is not iterable

DrillDownTagIds Range: 20-40
Sports: ['Tennis', 'Voetbal', 'Football', 'IJshockey']
Competitions: ['WTA Abu Dhabi, Enkelspel', 'WTA 125K Mumbai, Enkelspel', 'Duitsland DFB Pokal', 'Engeland EFL Trophy', 'VS NHL', 'WTA Abu Dhabi, Doubles', 'WTA Cluj Napoca, Doubles', 'Duitsland Bundesliga', 'Engeland Championship', 'Frankrijk Beker', 'Spanje LaLiga', 'Engeland FA Cup', 'Spanje Copa del Rey', 'Frankrijk Ligue 1', 'England League Cup', 'Engeland Premier League', 'Italië Coppa Italia', 'WTA 125K Mumbai, Doubles', 'WTA Cluj Napoca, Enkelspel', 'Spanje Segunda Division', 'NFL', 'Italië Serie A']

DrillDownTagIds Range: 40-60
Sports: ['Tennis', 'Voetbal', 'Football', 'IJshockey']
Competitions: ['WTA Abu Dhabi, Enkelspel', 'WTA 125K Mumbai, Enkels

In [6]:
# Optionally, save the range summary to a file
pd.DataFrame.from_dict(range_summary, orient="index").to_csv(f"range_summary_{datetime.utcnow()}.csv")

In [19]:
# keys_with_basketbal = [key for key, value in range_summary.items() if 'Voetbal' in value.get('sports', [])]
# keys_with_basketbal

In [64]:
final_matches_df = final_matches_df[final_matches_df['match_name'].str.contains('AS Saint-Etienne vs Stade Rennes')]

In [65]:
final_matches_df

,event_id,match_name,start_time,home_team,away_team,competition,country,sport,home_odds,away_odds
2674,6612005,AS Saint-Etienne vs Stade Rennes,2025-02-08T20:05:00Z,AS Saint-Etienne,Stade Rennes,Frankrijk Ligue 1,Frankrijk,Voetbal,3.2,2.17
2864,6612005,AS Saint-Etienne vs Stade Rennes,2025-02-08T20:05:00Z,AS Saint-Etienne,Stade Rennes,Frankrijk Ligue 1,Frankrijk,Voetbal,3.2,2.17
3054,6612005,AS Saint-Etienne vs Stade Rennes,2025-02-08T20:05:00Z,AS Saint-Etienne,Stade Rennes,Frankrijk Ligue 1,Frankrijk,Voetbal,3.2,2.17
3244,6612005,AS Saint-Etienne vs Stade Rennes,2025-02-08T20:05:00Z,AS Saint-Etienne,Stade Rennes,Frankrijk Ligue 1,Frankrijk,Voetbal,3.2,2.17
3435,6612005,AS Saint-Etienne vs Stade Rennes,2025-02-08T20:05:00Z,AS Saint-Etienne,Stade Rennes,Frankrijk Ligue 1,Frankrijk,Voetbal,3.2,2.17
3625,6612005,AS Saint-Etienne vs Stade Rennes,2025-02-08T20:05:00Z,AS Saint-Etienne,Stade Rennes,Frankrijk Ligue 1,Frankrijk,Voetbal,3.2,2.17
3815,6612005,AS Saint-Etienne vs Stade Rennes,2025-02-08T20:05:00Z,AS Saint-Etienne,Stade Rennes,Frankrijk Ligue 1,Frankrijk,Voetbal,3.2,2.17
4030,6612005,AS Saint-Etienne vs Stade Rennes,2025-02-08T20:05:00Z,AS Saint-Etienne,Stade Rennes,Frankrijk Ligue 1,Frankrijk,Voetbal,3.2,2.17
4220,6612005,AS Saint-Etienne vs Stade Rennes,2025-02-08T20:05:00Z,AS Saint-Etienne,Stade Rennes,Frankrijk Ligue 1,Frankrijk,Voetbal,3.2,2.17
4410,6612005,AS Saint-Etienne vs Stade Rennes,2025-02-08T20:05:00Z,AS Saint-Etienne,Stade Rennes,Frankrijk Ligue 1,Frankrijk,Voetbal,3.2,2.17


In [86]:
import requests
import time
import random
import pandas as pd

all_data = []
batch_size = 30

for i in range(0, len(final_matches_df['event_id']), batch_size):
    try:
        batch_ids = final_matches_df['event_id'][i:i + batch_size]
        event_ids_str = ','.join(map(str, batch_ids))

        url = (
            f"https://content.toto.nl/content-service/api/v1/q/events-by-ids?"
            f"eventIds={event_ids_str}&"
            "includeChildMarkets=true&includeCollections=true&includePriorityCollectionChildMarkets=true&"
            "includePriceHistory=true&includeCommentary=true&includeIncidents=true&includeRace=true&"
            "includePools=true&includeNonFixedOdds=true&lang=nl-NL&channel=I"
        )

        headers = {
            'accept': 'application/json',
            'accept-language': 'en-US,en;q=0.9,nl;q=0.8',
            'Cookie': 'incap_ses_1581_2280942=FvpNPI919nl1UsyZNNfwFR6GL2cAAAAA8yX5UHfYkaWD/ax05SY+Jg==; visid_incap_2280942=CzAo1Y6yTMa654jo8RfVRtBiL2cAAAAAQkIPAAAAAAAE7rpFtkP20364qTw5VQQQ'
        }

        response = requests.get(url, headers=headers)
        json_data = response.json()

        # Lijst om resultaten per event op te slaan
        event_market_data = []

        for event in json_data['data']['events']:
            event_id = event['id']
            event_name = event['name']

            # Filter de juiste marketIds uit de collectie met naam 'Alles'
            market_ids = []
            for collection in event.get('collections', []):
                if collection['name'] == 'Alles':
                    market_ids = collection.get('marketIds', [])
                    break  # Stop zodra we de juiste collectie vinden

            # Voeg alleen event_id en marketIds toe
            event_market_data.append({
                'event_id': event_id,
                'Event Name': event_name,
                'MarketIDs': market_ids
            })

        if event_market_data:
            temp_df = pd.DataFrame(event_market_data)
            all_data.append(temp_df)

    except Exception as e:
        print(f"Error: {e}")

if all_data:
    final_df = pd.concat(all_data, ignore_index=True)
    print(final_df)
else:
    print("No data found")

  event_id                        Event Name  \
0  6612005  AS Saint-Etienne vs Stade Rennes   

                                           MarketIDs  
0  [159429586, 159429587, 159429588, 159429737, 1...  


In [72]:
len(temp_df['MarketIDs'].iloc[0])

19

In [84]:
import requests
import time
import random
import pandas as pd

# Initialize an empty list to accumulate all data
all_data = []

# Set batch size
batch_size = 30

# Loop over event_ids in batches of 500
for i in range(0, len(final_matches_df['event_id']), batch_size):
    try:
        # Get a batch of event_ids
        batch_ids = final_matches_df['event_id'][i:i + batch_size]
        
        # Concatenate event_ids for the URL
        event_ids_str = ','.join(map(str, batch_ids))
        
        url = (
            "https://content.toto.nl/content-service/api/v1/q/events-by-ids?"
            "marketIds=159817403,159817408,159817409,159817435,159817436,159817437,159817438,159817440,159817441,159817442,159817443,159817444,159817451,159817456,159817462,159817466,159817471,159817474,159817475,159817476,159817477,159817481,159817482,159817483,159817484,159817485,159817486,159817487,159817488,159817491,159817493,159817494,159817497,159817499,159817503,159817507,159817509,159817517,159817520,159817523,159817524,159817527,159817544,159817549,159817550,159817551,159860158,159860161,159860162,159860164,159860166,159860167,159860168,159860169,159860170,159860172,159860173,159860174,159860175,159860176,159860177,159860179,159860180,159860181,159860183,159860184,159860185,159860186,159860188,159860189&"
            "includeChildMarkets=true&"
            "includePriceHistory=false&"
            "includeCommentary=false&"
            "includeMedia=false&"
            "includePoolsInfo=false&"
            "includeNonFixedOdds=false&"
            "lang=nl-NL&"
            "channel=I"
            )
        
        headers = {
            'accept': 'application/json',
            'accept-language': 'en-US,en;q=0.9,nl;q=0.8',
            'Cookie': 'incap_ses_1581_2280942=FvpNPI919nl1UsyZNNfwFR6GL2cAAAAA8yX5UHfYkaWD/ax05SY+Jg==; visid_incap_2280942=CzAo1Y6yTMa654jo8RfVRtBiL2cAAAAAQkIPAAAAAAAE7rpFtkP20364qTw5VQQQ'
        }
        
        # Send request
        response = requests.get(url, headers=headers)
        json_data = response.json()

        # Initialize lists to store the data for this batch
        event_ids = []
        event_names = []
        market_names = []
        outcome_names = []
        odds_decimal = []
        prices_numerator = []
        prices_denominator = []
        outcome_types = []
        outcome_subtypes = []

        # Process the JSON response for each event
        for event in json_data['data']['events']:  # Iterate over each event
            event_id = event['id']
            event_name = event['name']
            
            for market in event['markets']:  # Iterate over each market in the event
                market_name = market['name']
                
                for outcome in market['outcomes']:  # Iterate over outcomes for each market
                    outcome_name = outcome['name']
                    odds_decimal_value = outcome['prices'][0]['decimal']
                    numerator = outcome['prices'][0]['numerator']
                    denominator = outcome['prices'][0]['denominator']
                    outcome_type = outcome['type']
                    outcome_subtype = outcome['subType']
                    
                    # Append captured values to the respective lists
                    event_ids.append(event_id)
                    event_names.append(event_name)
                    market_names.append(market_name)
                    outcome_names.append(outcome_name)
                    odds_decimal.append(odds_decimal_value)
                    prices_numerator.append(numerator)
                    prices_denominator.append(denominator)
                    outcome_types.append(outcome_type)
                    outcome_subtypes.append(outcome_subtype)

        # Create a temporary DataFrame for this batch and append it to all_data
        temp_df = pd.DataFrame({
            'event_id': event_ids,
            'Event Name': event_names,
            'Market Name': market_names,
            'Outcome Name': outcome_names,
            'Odds (Decimal)': odds_decimal,
            'Price Numerator': prices_numerator,
            'Price Denominator': prices_denominator,
            'Outcome Type': outcome_types,
            'Outcome SubType': outcome_subtypes
        })
        
        all_data.append(temp_df)
        
        # Wait between .. and .. seconds
        time.sleep(random.uniform(0.02, 1.81))
    except:
        None

# Concatenate all individual DataFrames into a final DataFrame
final_df = pd.concat(all_data, ignore_index=True)

# Show the final DataFrame
print(final_df)

    event_id                Event Name  \
0    6608244  Strasbourg vs Angers SCO   
1    6608244  Strasbourg vs Angers SCO   
2    6608244  Strasbourg vs Angers SCO   
3    6608244  Strasbourg vs Angers SCO   
4    6608244  Strasbourg vs Angers SCO   
..       ...                       ...   
276  6608244  Strasbourg vs Angers SCO   
277  6608244  Strasbourg vs Angers SCO   
278  6608244  Strasbourg vs Angers SCO   
279  6608244  Strasbourg vs Angers SCO   
280  6608244  Strasbourg vs Angers SCO   

                                     Market Name  \
0                Resultaat en Beide Teams Scoren   
1                Resultaat en Beide Teams Scoren   
2                Resultaat en Beide Teams Scoren   
3                Resultaat en Beide Teams Scoren   
4                Resultaat en Beide Teams Scoren   
..                                           ...   
276  Dubbele Kans en Aantal Goals Over/Under 4.5   
277  Dubbele Kans en Aantal Goals Over/Under 4.5   
278  Dubbele Kans en Aantal

In [85]:
final_df

,event_id,Event Name,Market Name,Outcome Name,Odds (Decimal),Price Numerator,Price Denominator,Outcome Type,Outcome SubType
0,6608244,Strasbourg vs Angers SCO,Resultaat en Beide Teams Scoren,Angers SCO en Nee,7.50,13,2,--,None
1,6608244,Strasbourg vs Angers SCO,Resultaat en Beide Teams Scoren,Angers SCO en Ja,8.50,15,2,--,None
2,6608244,Strasbourg vs Angers SCO,Resultaat en Beide Teams Scoren,Gelijkspel en Nee,10.00,9,1,--,None
3,6608244,Strasbourg vs Angers SCO,Resultaat en Beide Teams Scoren,Strasbourg en Ja,3.65,53,20,--,None
4,6608244,Strasbourg vs Angers SCO,Resultaat en Beide Teams Scoren,Gelijkspel en Ja,4.50,7,2,--,None
...,...,...,...,...,...,...,...,...,...
276,6608244,Strasbourg vs Angers SCO,Dubbele Kans en Aantal Goals Over/Under 4.5,Gelijkspel of Angers SCO en Over,19.00,18,1,--,None
277,6608244,Strasbourg vs Angers SCO,Dubbele Kans en Aantal Goals Over/Under 4.5,Strasbourg of Gelijkspel en Over,7.00,6,1,--,None
278,6608244,Strasbourg vs Angers SCO,Dubbele Kans en Aantal Goals Over/Under 4.5,Gelijkspel of Angers SCO en Under,1.91,91,100,--,None
279,6608244,Strasbourg vs Angers SCO,Dubbele Kans en Aantal Goals Over/Under 4.5,Strasbourg of Angers SCO en Over,6.25,21,4,--,None


In [82]:
url = (
    "https://content.toto.nl/content-service/api/v1/q/events-by-ids?"
    "marketIds=159817403,159817408,159817409,159817435,159817436,159817437,159817438,159817440,159817441,159817442,159817443,159817444,159817451,159817456,159817462,159817466,159817471,159817474,159817475,159817476,159817477,159817481,159817482,159817483,159817484,159817485,159817486,159817487,159817488,159817491,159817493,159817494,159817497,159817499,159817503,159817507,159817509,159817517,159817520,159817523,159817524,159817527,159817544,159817549,159817550,159817551,159860158,159860161,159860162,159860164,159860166,159860167,159860168,159860169,159860170,159860172,159860173,159860174,159860175,159860176,159860177,159860179,159860180,159860181,159860183,159860184,159860185,159860186,159860188,159860189&"
    "includeChildMarkets=true&"
    "includePriceHistory=false&"
    "includeCommentary=false&"
    "includeMedia=false&"
    "includePoolsInfo=false&"
    "includeNonFixedOdds=false&"
    "lang=nl-NL&"
    "channel=I"
    )

headers = {
    'accept': 'application/json',
    'accept-language': 'en-US,en;q=0.9,nl;q=0.8',
    'Cookie': 'incap_ses_1581_2280942=FvpNPI919nl1UsyZNNfwFR6GL2cAAAAA8yX5UHfYkaWD/ax05SY+Jg==; visid_incap_2280942=CzAo1Y6yTMa654jo8RfVRtBiL2cAAAAAQkIPAAAAAAAE7rpFtkP20364qTw5VQQQ'
}

# Send request
response = requests.get(url, headers=headers)
json_data = response.json()

In [100]:
final_matches_df = final_matches_df.drop_duplicates()

In [109]:
import requests
import time
import random
import pandas as pd

# Initialize an empty list to accumulate all data
all_data = []
batch_size = 10

# Stap 1: Haal marketIds op voor elk event
event_market_ids_dict = {}

for i in range(0, len(final_matches_df['event_id']), batch_size):
    try:
        batch_ids = final_matches_df['event_id'][i:i + batch_size]
        event_ids_str = ','.join(map(str, batch_ids))

        url = (
            f"https://content.toto.nl/content-service/api/v1/q/events-by-ids?"
            f"eventIds={event_ids_str}&"
            "includeChildMarkets=true&includeCollections=true&includePriorityCollectionChildMarkets=true&"
            "includePriceHistory=true&includeCommentary=true&includeIncidents=true&includeRace=true&"
            "includePools=true&includeNonFixedOdds=true&lang=nl-NL&channel=I"
        )

        headers = {
            'accept': 'application/json',
            'accept-language': 'en-US,en;q=0.9,nl;q=0.8',
            'Cookie': 'incap_ses_1581_2280942=FvpNPI919nl1UsyZNNfwFR6GL2cAAAAA8yX5UHfYkaWD/ax05SY+Jg==; visid_incap_2280942=CzAo1Y6yTMa654jo8RfVRtBiL2cAAAAAQkIPAAAAAAAE7rpFtkP20364qTw5VQQQ'
        }

        response = requests.get(url, headers=headers)
        json_data = response.json()

        for event in json_data['data']['events']:
            event_id = event['id']

            # Haal alleen de marketIds op van collectie 'Alles'
            market_ids = []
            for collection in event.get('collections', []):
                if collection['name'] == 'Alles':
                    market_ids = collection.get('marketIds', [])
                    break

            # Opslaan in dictionary
            event_market_ids_dict[event_id] = market_ids

        # time.sleep(random.uniform(0.02, 1.81))

    except Exception as e:
        print(f"Error bij ophalen van marketIds: {e}. Bij event_ids: {batch_ids}")

# Stap 2: Gebruik de opgehaalde marketIds in de tweede request
for event_id, market_ids in event_market_ids_dict.items():
    if not market_ids:
        continue  # Sla over als er geen marketIds zijn

    market_ids_str = ','.join(map(str, market_ids))

    url = (
        f"https://content.toto.nl/content-service/api/v1/q/events-by-ids?"
        f"marketIds={market_ids_str}&"
        "includeChildMarkets=true&"
        "includePriceHistory=false&"
        "includeCommentary=false&"
        "includeMedia=false&"
        "includePoolsInfo=false&"
        "includeNonFixedOdds=false&"
        "lang=nl-NL&"
        "channel=I"
    )

    try:
        response = requests.get(url, headers=headers)
        json_data = response.json()

        event_ids = []
        event_names = []
        market_names = []
        outcome_names = []
        odds_decimal = []
        prices_numerator = []
        prices_denominator = []
        outcome_types = []
        outcome_subtypes = []

        for event in json_data['data']['events']:
            event_name = event['name']

            for market in event['markets']:
                market_name = market['name']

                for outcome in market['outcomes']:
                    outcome_name = outcome['name']
                    odds_decimal_value = outcome['prices'][0]['decimal']
                    numerator = outcome['prices'][0]['numerator']
                    denominator = outcome['prices'][0]['denominator']
                    outcome_type = outcome['type']
                    outcome_subtype = outcome['subType']

                    event_ids.append(event_id)
                    event_names.append(event_name)
                    market_names.append(market_name)
                    outcome_names.append(outcome_name)
                    odds_decimal.append(odds_decimal_value)
                    prices_numerator.append(numerator)
                    prices_denominator.append(denominator)
                    outcome_types.append(outcome_type)
                    outcome_subtypes.append(outcome_subtype)

        temp_df = pd.DataFrame({
            'event_id': event_ids,
            'Event Name': event_names,
            'Market Name': market_names,
            'Outcome Name': outcome_names,
            'Odds (Decimal)': odds_decimal,
            'Price Numerator': prices_numerator,
            'Price Denominator': prices_denominator,
            'Outcome Type': outcome_types,
            'Outcome SubType': outcome_subtypes
        })

        all_data.append(temp_df)

        # time.sleep(random.uniform(0.02, 1.81))

    except Exception as e:
        print(f"Error bij ophalen van odds data: {e}")

# Stap 3: Combineer alle opgehaalde data
if all_data:
    final_df = pd.concat(all_data, ignore_index=True)
    print(final_df)
else:
    print("No data found")

Error bij ophalen van marketIds: 'NoneType' object is not iterable. Bij event_ids: 10    6560542
11    6632909
12    6517346
13    6559730
14    6512573
15    6706138
16    6563560
17    6680339
18    6680344
19    6608243
Name: event_id, dtype: object


KeyboardInterrupt: 

In [ ]:
import requests
import pandas as pd
from typing import Dict, List
import time
import random

def get_headers() -> Dict[str, str]:
    return {
        'accept': 'application/json',
        'accept-language': 'en-US,en;q=0.9,nl;q=0.8',
        'Cookie': 'incap_ses_1581_2280942=FvpNPI919nl1UsyZNNfwFR6GL2cAAAAA8yX5UHfYkaWD/ax05SY+Jg==; visid_incap_2280942=CzAo1Y6yTMa654jo8RfVRtBiL2cAAAAAQkIPAAAAAAAE7rpFtkP20364qTw5VQQQ'
    }

def collect_market_ids(event_ids: List[int], batch_size: int = 6) -> Dict[int, List[int]]:
    """Collect market IDs for all events in batches"""
    event_market_ids_dict = {}
    headers = get_headers()
    
    for i in range(0, len(event_ids), batch_size):
        batch_ids = event_ids[i:i + batch_size]
        event_ids_str = ','.join(map(str, batch_ids))
        
        url = (
            "https://content.toto.nl/content-service/api/v1/q/events-by-ids?"
            f"eventIds={event_ids_str}&"
            "includeChildMarkets=true&includeCollections=true&"
            "includePriorityCollectionChildMarkets=true&includePriceHistory=true&"
            "includeCommentary=true&includeIncidents=true&includeRace=true&"
            "includePools=true&includeNonFixedOdds=true&lang=nl-NL&channel=I"
        )
        
        try:
            response = requests.get(url, headers=headers)
            json_data = response.json()
            
            for event in json_data['data']['events']:
                event_id = event['id']
                market_ids = []
                for collection in event.get('collections', []):
                    if collection['name'] == 'Alles' or collection['name'] == 'UNASSIGNED"':
                        market_ids = collection.get('marketIds', [])
                        break
                event_market_ids_dict[event_id] = market_ids
            
        except Exception as e:
            print(f"Error collecting market IDs: {e} for event_ids: {batch_ids}")
    
    return event_market_ids_dict

def process_market_data(market_ids: List[int], batch_size: int = 50) -> pd.DataFrame:
    """Process market data in batches and return a DataFrame"""
    all_data = []
    headers = get_headers()
    
    for i in range(0, len(market_ids), batch_size):
        batch_market_ids = market_ids[i:i + batch_size]
        market_ids_str = ','.join(map(str, batch_market_ids))
        
        url = (
            "https://content.toto.nl/content-service/api/v1/q/events-by-ids?"
            f"marketIds={market_ids_str}&"
            "includeChildMarkets=true&includePriceHistory=false&"
            "includeCommentary=false&includeMedia=false&"
            "includePoolsInfo=false&includeNonFixedOdds=false&"
            "lang=nl-NL&channel=I"
        )
        
        try:
            response = requests.get(url, headers=headers)
            json_data = response.json()
            
            batch_data = []
            for event in json_data['data']['events']:
                event_id = event['id']
                event_name = event['name']
                
                for market in event['markets']:
                    for outcome in market['outcomes']:
                        batch_data.append({
                            'event_id': event_id,
                            'Event Name': event_name,
                            'Market Name': market['name'],
                            'Outcome Name': outcome['name'],
                            'Odds (Decimal)': outcome['prices'][0]['decimal'],
                            'Price Numerator': outcome['prices'][0]['numerator'],
                            'Price Denominator': outcome['prices'][0]['denominator'],
                            'Outcome Type': outcome['type'],
                            'Outcome SubType': outcome['subtype']
                        })
            
            if batch_data:
                all_data.extend(batch_data)
            
            # time.sleep(random.uniform(0.02, 1.81))
            
        except Exception as e:
            print(f"Error processing market data: {e} for batch: {batch_market_ids}")
    
    return pd.DataFrame(all_data) if all_data else pd.DataFrame()

def main(event_ids: List[int]):
    # Step 1: Collect all market IDs
    event_market_ids_dict = collect_market_ids(event_ids)
    
    # Step 2: Flatten market IDs into a single list
    all_market_ids = []
    for market_ids in event_market_ids_dict.values():
        all_market_ids.extend(market_ids)
    
    # Step 3: Process all market data
    if all_market_ids:
        final_df = process_market_data(all_market_ids)
        return final_df
    else:
        print("No market IDs found")
        return pd.DataFrame()

# Usage example:
if __name__ == "__main__":
    # Replace with your list of event IDs
    event_ids = final_matches_df['event_id'].tolist()
    result_df = main(event_ids)
    print(result_df)

Error processing market data: 'subtype' for batch: ['158318877', '158573034', '158573035', '158573036', '158573037', '158573038', '158573077', '158573078', '158573079', '158573080', '158573081', '158573082', '158573083', '158573084', '158573085', '158573086', '158573087', '158573088', '158573089', '158573090', '158573091', '158573092', '158573093', '158573094', '158573095', '158573096', '158573097', '158573098', '158573099', '158573100', '158573101', '158573102', '158573103', '158573104', '158573105', '158573106', '158573107', '158573108', '158573109', '158573110', '158573111', '158573112', '158573113', '158573114', '158573115', '158573116', '158573117', '158573118', '158573119', '158573120']
Error processing market data: 'subtype' for batch: ['158573121', '158573122', '158573123', '158573124', '158573125', '158573126', '158573127', '158573128', '158573129', '158573130', '158573131', '158573132', '158573133', '158573134', '158573135', '158573136', '158573137', '158573138', '158573139',

In [131]:
event_market_ids_dict = collect_market_ids(event_ids)

Error collecting market IDs: 'NoneType' object is not iterable for event_ids: ['6611772', '6514423', '5951242', '6632905', '6632904', '6692173', '6296228', '6681132', '6563559', '6296285', '6560542', '6632909', '6517346', '6559730', '6512573', '6706138', '6563560', '6680339', '6680344', '6608243', '6680342', '6406230', '6736535', '6680346', '6689679', '6559940', '6736587', '6689683', '6680343', '6680413', '6680353', '6680341', '6680347', '6680345', '6680354', '6748513', '6748507', '6748504', '6748506', '6748511', '6748510', '6748508', '6748514', '6748509', '6748505', '6605181', '6609447', '6581513', '6559732', '6580693']
Error collecting market IDs: 'NoneType' object is not iterable for event_ids: ['6559715', '6559681', '6559718', '6581514', '6559731', '6559713', '6581515', '6559714', '6019547', '6019534', '6019552', '6019541', '6019538', '6019549', '6019550', '6019542', '6019544', '6019540', '6088963', '6088971', '6088961', '6088954', '6088973', '6088970', '6088979', '6088975', '60889

KeyboardInterrupt: 

In [6]:
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import Dict, List, Any
import pandas as pd
import time
import random

def get_headers() -> Dict[str, str]:
    return {
        'accept': 'application/json',
        'accept-language': 'en-US,en;q=0.9,nl;q=0.8',
        'Cookie': 'incap_ses_1581_2280942=FvpNPI919nl1UsyZNNfwFR6GL2cAAAAA8yX5UHfYkaWD/ax05SY+Jg==; visid_incap_2280942=CzAo1Y6yTMa654jo8RfVRtBiL2cAAAAAQkIPAAAAAAAE7rpFtkP20364qTw5VQQQ'
    }

def fetch_market_ids_for_event_batch(batch_ids: List[int]) -> Dict[int, List[int]]:
    """
    Fetch market IDs for a small batch of event IDs
    
    Args:
        batch_ids (List[int]): List of event IDs to process
    
    Returns:
        Dict[int, List[int]]: Dictionary mapping event IDs to their market IDs
    """
    headers = get_headers()
    event_market_ids_dict = {}
    
    event_ids_str = ','.join(map(str, batch_ids))
    
    url = (
        "https://content.toto.nl/content-service/api/v1/q/events-by-ids?"
        f"eventIds={event_ids_str}&"
        "includeChildMarkets=true&includeCollections=true&"
        "includePriorityCollectionChildMarkets=true&includePriceHistory=true&"
        "includeCommentary=true&includeIncidents=true&includeRace=true&"
        "includePools=true&includeNonFixedOdds=true&lang=nl-NL&channel=I"
    )
    
    try:
        # Add random sleep to reduce request rate
        time.sleep(random.uniform(0.1, 0.3))
        
        response = requests.get(url, headers=headers, timeout=30)
        json_data = response.json()
        
        for event in json_data['data']['events']:
            event_id = event['id']
            market_ids = []
            for collection in event.get('collections', []):
                if collection['name'] in ['Alles', 'UNASSIGNED']:
                    market_ids = collection.get('marketIds', [])
                    break
            event_market_ids_dict[event_id] = market_ids
        
    except Exception as e:
        print(f"Error collecting market IDs: {e} for event_ids: {batch_ids}")
    
    return event_market_ids_dict

def collect_market_ids(event_ids: List[int], max_workers: int = 10, batch_size: int = 7) -> Dict[int, List[int]]:
    """
    Concurrently collect market IDs for all events
    
    Args:
        event_ids (List[int]): Full list of event IDs
        max_workers (int): Maximum number of concurrent threads
        batch_size (int): Number of event IDs to process in each batch
    
    Returns:
        Dict[int, List[int]]: Comprehensive dictionary of event IDs to market IDs
    """
    event_market_ids_dict = {}
    
    # Split event_ids into batches
    event_id_batches = [event_ids[i:i + batch_size] for i in range(0, len(event_ids), batch_size)]
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit all batches
        future_to_batch = {
            executor.submit(fetch_market_ids_for_event_batch, batch): batch 
            for batch in event_id_batches
        }
        
        # Collect results
        for future in as_completed(future_to_batch):
            try:
                batch_results = future.result()
                event_market_ids_dict.update(batch_results)
                print(f"Processed batch, total events so far: {len(event_market_ids_dict)}")
            except Exception as e:
                print(f"Batch processing error: {e}")
    
    return event_market_ids_dict


In [ ]:
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import Dict, List, Any
import pandas as pd
import time
import random

def get_headers() -> Dict[str, str]:
    return {
        'accept': 'application/json',
        'accept-language': 'en-US,en;q=0.9,nl;q=0.8',
        'Cookie': 'incap_ses_1581_2280942=FvpNPI919nl1UsyZNNfwFR6GL2cAAAAA8yX5UHfYkaWD/ax05SY+Jg==; visid_incap_2280942=CzAo1Y6yTMa654jo8RfVRtBiL2cAAAAAQkIPAAAAAAAE7rpFtkP20364qTw5VQQQ'
    }

def fetch_market_data_for_batch(batch_market_ids: List[int]) -> List[Dict[str, Any]]:
    """
    Fetch market data for a batch of market IDs
    
    Args:
        batch_market_ids (List[int]): List of market IDs to process
    
    Returns:
        List[Dict[str, Any]]: List of extracted market data
    """
    headers = get_headers()
    batch_data = []
    
    market_ids_str = ','.join(map(str, batch_market_ids))
    
    url = (
        "https://content.toto.nl/content-service/api/v1/q/events-by-ids?"
        f"marketIds={market_ids_str}&"
        "includeChildMarkets=true&includePriceHistory=false&"
        "includeCommentary=false&includeMedia=false&"
        "includePoolsInfo=false&includeNonFixedOdds=false&"
        "lang=nl-NL&channel=I"
    )
    
    try:
        # Add random sleep to reduce request rate
        time.sleep(random.uniform(0.5, 2))
        
        response = requests.get(url, headers=headers, timeout=30)
        json_data = response.json()
        
        for event in json_data['data']['events']:
            event_id = event['id']
            event_name = event['name']
            
            for market in event['markets']:
                for outcome in market['outcomes']:
                    batch_data.append({
                        'event_id': event_id,
                        'Event Name': event_name,
                        'Market Name': market['name'],
                        'Outcome Name': outcome['name'],
                        'Odds (Decimal)': outcome['prices'][0]['decimal'],
                        'Price Numerator': outcome['prices'][0]['numerator'],
                        'Price Denominator': outcome['prices'][0]['denominator'],
                        'Outcome Type': outcome['type'],
                        'Outcome SubType': outcome.get('subType', '')
                    })
        
    except Exception as e:
        print(f"Error processing market data: {e} for batch: {batch_market_ids}")
    
    return batch_data

def process_market_data(
    market_ids: List[int], 
    max_workers: int = 20, 
    batch_size: int = 7
) -> pd.DataFrame:
    """
    Concurrently process market data for all market IDs
    
    Args:
        market_ids (List[int]): Full list of market IDs
        max_workers (int): Maximum number of concurrent threads
        batch_size (int): Number of market IDs to process in each batch
    
    Returns:
        pd.DataFrame: DataFrame containing processed market data
    """
    # Split market IDs into batches
    market_id_batches = [market_ids[i:i + batch_size] for i in range(0, len(market_ids), batch_size)]
    
    # Collected data from all batches
    all_data = []
    
    # Use ThreadPoolExecutor for concurrent processing
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit all batches
        future_to_batch = {
            executor.submit(fetch_market_data_for_batch, batch): batch 
            for batch in market_id_batches
        }
        
        # Collect results
        for future in as_completed(future_to_batch):
            try:
                batch_results = future.result()
                all_data.extend(batch_results)
            except Exception as e:
                print(f"Batch processing error: {e}")
    
    # Convert to DataFrame
    return pd.DataFrame(all_data) if all_data else pd.DataFrame()

def main(event_ids: List[int]) -> pd.DataFrame:
    """
    Main function to process market data for given event IDs
    
    Args:
        event_ids (List[int]): List of event IDs to process
    
    Returns:
        pd.DataFrame: Processed market data
    """
    # Step 1: Collect all market IDs
    event_market_ids_dict = collect_market_ids(
        event_ids, 
        max_workers=100,  # Adjust based on system capabilities
        batch_size=1     # Keep batch size small
    )
    
    # Step 2: Flatten market IDs into a single list
    all_market_ids = []
    for market_ids in event_market_ids_dict.values():
        all_market_ids.extend(market_ids)
    
    # Step 3: Process all market data
    if all_market_ids:
        final_df = process_market_data(
            all_market_ids, 
            max_workers=100,  # Adjust based on system capabilities
            batch_size=100     # Keep batch size small
        )
        return final_df
    else:
        print("No market IDs found")
        return pd.DataFrame()

# Usage example:
if __name__ == "__main__":
    # Assuming final_matches_df is defined with 'event_id' column
    event_ids = final_matches_df['event_id'].tolist()
    result_df = main(event_ids)
    print(result_df)

Processed batch, total events so far: 1
Processed batch, total events so far: 2
Processed batch, total events so far: 3
Processed batch, total events so far: 4
Processed batch, total events so far: 5
Processed batch, total events so far: 6
Processed batch, total events so far: 7
Processed batch, total events so far: 8
Processed batch, total events so far: 9
Processed batch, total events so far: 10
Processed batch, total events so far: 11
Processed batch, total events so far: 12
Processed batch, total events so far: 13
Processed batch, total events so far: 14
Processed batch, total events so far: 15
Processed batch, total events so far: 16
Processed batch, total events so far: 17
Processed batch, total events so far: 18
Processed batch, total events so far: 19
Processed batch, total events so far: 20
Processed batch, total events so far: 21
Processed batch, total events so far: 22
Processed batch, total events so far: 23
Processed batch, total events so far: 24
Processed batch, total ev

In [9]:
result_df

,event_id,Event Name,Market Name,Outcome Name,Odds (Decimal),Price Numerator,Price Denominator,Outcome Type,Outcome SubType
0,6672946,Brooklyn Nets vs Washington Wizards,Winnaar,Washington Wizards,2.05,21,20,HH,A
1,6672946,Brooklyn Nets vs Washington Wizards,Winnaar,Brooklyn Nets,1.80,4,5,HH,H
2,6672946,Brooklyn Nets vs Washington Wizards,Handicap - Winnaar 1.5,Washington Wizards,2.10,11,10,WH,A
3,6672946,Brooklyn Nets vs Washington Wizards,Handicap - Winnaar 1.5,Brooklyn Nets,1.63,63,100,WH,H
4,6672946,Brooklyn Nets vs Washington Wizards,Handicap - Winnaar -3.5,Washington Wizards,1.67,67,100,WH,A
...,...,...,...,...,...,...,...,...,...
298003,6667544,Yoenis Tellez vs Julian Williams,Winnaar,Julian Williams,7.75,27,4,HH,A
298004,6317893,Nick Ball vs Terrence John Doheny,Winnaar,Nick Ball,1.06,3,50,HH,H
298005,6317893,Nick Ball vs Terrence John Doheny,Winnaar,Terrence John Doheny,7.00,6,1,HH,A
298006,6704289,Viddal Riley vs Cheavan Clarke,Winnaar,Viddal Riley,1.63,63,100,HH,H


In [10]:
# Perform the left join on 'event_id' to get 'sport' and 'group_name' from events_df
final_df = result_df.merge(final_matches_df[['event_id', 'sport', 'competition']], on='event_id', how='left').drop_duplicates()

In [22]:
final_df[(final_df['Market Name'].str.contains('Scoort'))]#&(final_df['Outcome Name']=='Over')]

,event_id,Event Name,Market Name,Outcome Name,Odds (Decimal),Price Numerator,Price Denominator,Outcome Type,Outcome SubType,sport,competition
115,6617665,Rakow Czestochowa vs GKS Katowice,GKS Katowice Scoort in Beide Helften,Nee,1.08,2,25,--,None,Voetbal,Polen Ekstraklasa
116,6617665,Rakow Czestochowa vs GKS Katowice,GKS Katowice Scoort in Beide Helften,Ja,6.25,21,4,--,None,Voetbal,Polen Ekstraklasa
223,6617665,Rakow Czestochowa vs GKS Katowice,Resultaat en Scoort Eerste Goal,GKS Katowice Wint en Rakow Czestochowa Scoort ...,29.00,28,1,--,None,Voetbal,Polen Ekstraklasa
224,6617665,Rakow Czestochowa vs GKS Katowice,Resultaat en Scoort Eerste Goal,Gelijkspel en Rakow Czestochowa Scoort Eerste ...,9.50,17,2,--,None,Voetbal,Polen Ekstraklasa
225,6617665,Rakow Czestochowa vs GKS Katowice,Resultaat en Scoort Eerste Goal,Rakow Czestochowa Wint en GKS Katowice Scoort ...,10.00,9,1,--,None,Voetbal,Polen Ekstraklasa
...,...,...,...,...,...,...,...,...,...,...,...
355657,6710514,Real Esteli FC vs Tigres Uanl,Real Esteli FC Scoort in Beide Helften,Ja,4.25,13,4,--,None,Voetbal,CONCACAF Champions Cup
355834,6765041,USD Sestri Levante 1919 vs US Pianese,US Pianese Scoort in Beide Helften,Nee,1.22,11,50,--,None,Voetbal,Italy Serie C
355835,6765041,USD Sestri Levante 1919 vs US Pianese,US Pianese Scoort in Beide Helften,Ja,3.25,9,4,--,None,Voetbal,Italy Serie C
355869,6765041,USD Sestri Levante 1919 vs US Pianese,USD Sestri Levante 1919 Scoort in Beide Helften,Ja,4.25,13,4,--,None,Voetbal,Italy Serie C


In [24]:
final_df[(final_df['Market Name'].str.contains('Scoort'))&~(final_df['Market Name'].str.contains('en'))]

,event_id,Event Name,Market Name,Outcome Name,Odds (Decimal),Price Numerator,Price Denominator,Outcome Type,Outcome SubType,sport,competition
3655,6749458,Milton Keynes Dons vs Bromley FC,Speler Scoort Twee Of Meer,Joe White,67.0,66,1,MG,H,Voetbal,Engeland League 2
3656,6749458,Milton Keynes Dons vs Bromley FC,Speler Scoort Twee Of Meer,Marcus Dinanga,18.0,17,1,MG,A,Voetbal,Engeland League 2
3657,6749458,Milton Keynes Dons vs Bromley FC,Speler Scoort Twee Of Meer,Besart Topalloj,201.0,200,1,MG,A,Voetbal,Engeland League 2
3658,6749458,Milton Keynes Dons vs Bromley FC,Speler Scoort Twee Of Meer,Jay Williams,126.0,125,1,MG,H,Voetbal,Engeland League 2
3659,6749458,Milton Keynes Dons vs Bromley FC,Speler Scoort Twee Of Meer,Brooklyn Ilunga,91.0,90,1,MG,A,Voetbal,Engeland League 2
...,...,...,...,...,...,...,...,...,...,...,...
328680,6758247,Independiente Rivadavia De Mendoza vs Estudian...,2e Helft - Speler Scoort 3 Goals of Meer,Maximiliano Gonzalo Alvez,91.0,90,1,HS,H,Voetbal,Argentinië Superliga
328682,6758247,Independiente Rivadavia De Mendoza vs Estudian...,2e Helft - Speler Scoort 3 Goals of Meer,Javier Altamirano,91.0,90,1,HS,A,Voetbal,Argentinië Superliga
328684,6758247,Independiente Rivadavia De Mendoza vs Estudian...,2e Helft - Speler Scoort 3 Goals of Meer,Edwuin Cetre,81.0,80,1,HS,A,Voetbal,Argentinië Superliga
328686,6758247,Independiente Rivadavia De Mendoza vs Estudian...,1e Helft - Speler Scoort 3 Goals of Meer,Guido Carrillo,91.0,90,1,HS,A,Voetbal,Argentinië Superliga


In [28]:
import requests

def get_drilldown_football_nodes():
    """
    Fetch the drilldown tree and navigate to football nodes
    
    Returns:
        list: List of football-related drilldown nodes
    """
    url = "https://content.toto.nl/content-service/api/v1/q/drilldown-tree"
    
    # Query parameters
    params = {
        "drilldownNodeIds": "2",
        "eventState": "OPEN_EVENT",
        "lang": "nl-NL",
        "channel": "I"
    }
    
    # Headers from your request
    headers = {
        'accept': 'application/json',
        'accept-language': 'en-US,en;q=0.5',
        'origin': 'https://sport.toto.nl',
        'referer': 'https://sport.toto.nl/'
    }
    
    try:
        response = requests.get(url, params=params, headers=headers)
        response.raise_for_status()
        
        data = response.json()
        
        # Navigate through the response structure
        # First find node with id 2
        root_nodes = data['data']['drilldownNodes']
        sports_node = next((node for node in root_nodes if node['id'] == '2'), None)
        
        if sports_node:
            # Then find the 'Voetbal' node
            football_node = next((node for node in sports_node['drilldownNodes'] 
                                if node['name'] == 'Voetbal'), None)
            
            if football_node:
                return football_node['drilldownNodes']
            else:
                print("Voetbal node not found")
                return None
        else:
            print("Sports node (id: 2) not found")
            return root_nodes
            
    except Exception as e:
        print(f"Error fetching drilldown tree: {e}")
        return None

# Usage example
if __name__ == "__main__":
    football_nodes = get_drilldown_football_nodes()
    if football_nodes:
        print("\nFound football nodes:")
        for node in football_nodes:
            print(f"- {node['name']} (ID: {node['id']})")


Found football nodes:
- Nederland (ID: 1052)
- Engeland (ID: 40)
- Spanje (ID: 44)
- Italië (ID: 43)
- Duitsland (ID: 42)
- Frankrijk (ID: 41)
- België (ID: 864)
- Portugal (ID: 551)
- Turkije (ID: 537)
- Saudi Arabië (ID: 987)
- VS (ID: 595)
- Europees (ID: 3214)
- Wereldwijd (ID: 667)
- Americas (ID: 9342)
- Africa (ID: 6505)
- Andorra (ID: 4578)
- Algerije (ID: 1797)
- Argentinië (ID: 1670)
- Australië (ID: 825)
- Azerbeidzjan (ID: 786)
- Bahrein (ID: 1000)
- Bosnia and Herzegovina (ID: 963)
- Brazilië (ID: 896)
- Bulgarije (ID: 619)
- Burundi (ID: 519)
- Chili (ID: 1132)
- Cambodja (ID: 4125)
- Costa Rica (ID: 475)
- Colombia (ID: 1225)
- Cyprus (ID: 1068)
- Denemarken (ID: 479)
- Egypte (ID: 994)
- El Salvador (ID: 1607)
- Ethiopia (ID: 3396)
- Finland (ID: 692)
- Filipijnen (ID: 5317)
- Guatemala (ID: 1548)
- Griekenland (ID: 513)
- Honduras (ID: 1471)
- Hong Kong (ID: 1353)
- Hongarije (ID: 484)
- IJsland (ID: 617)
- Ierland (ID: 925)
- Indonesia (ID: 3528)
- India (ID: 1799)
-

In [29]:
import re

output_text = """Found football nodes: - Nederland (ID: 1052) - Engeland (ID: 40) - Spanje (ID: 44) - Italië (ID: 43) - Duitsland (ID: 42) - Frankrijk (ID: 41) - België (ID: 864) - Portugal (ID: 551) - Turkije (ID: 537) - Saudi Arabië (ID: 987) - VS (ID: 595) - Europees (ID: 3214) - Wereldwijd (ID: 667) - Americas (ID: 9342) - Africa (ID: 6505) - Andorra (ID: 4578) - Algerije (ID: 1797) - Argentinië (ID: 1670) - Australië (ID: 825) - Azerbeidzjan (ID: 786) - Bahrein (ID: 1000) - Bosnia and Herzegovina (ID: 963) - Brazilië (ID: 896) - Bulgarije (ID: 619) - Burundi (ID: 519) - Chili (ID: 1132) - Cambodja (ID: 4125) - Costa Rica (ID: 475) - Colombia (ID: 1225) - Cyprus (ID: 1068) - Denemarken (ID: 479) - Egypte (ID: 994) - El Salvador (ID: 1607) - Ethiopia (ID: 3396) - Finland (ID: 692) - Filipijnen (ID: 5317) - Guatemala (ID: 1548) - Griekenland (ID: 513) - Honduras (ID: 1471) - Hong Kong (ID: 1353) - Hongarije (ID: 484) - IJsland (ID: 617) - Ierland (ID: 925) - Indonesia (ID: 3528) - India (ID: 1799) - Israël (ID: 542) - Iran (ID: 795) - Japan (ID: 581) - Jordanië (ID: 988) - Koeweit (ID: 1022) - Kroatië (ID: 544) - Luxemburg (ID: 1355) - Malta (ID: 1590) - Maleisië (ID: 3143) - Mexico (ID: 888) - Marokko (ID: 919) - Nicaragua (ID: 970) - Noord-Ierland (ID: 915) - Noorwegen (ID: 583) - Oekraine (ID: 501) - Oman (ID: 1645) - Oostenrijk (ID: 534) - Panama (ID: 3013) - Paraguay (ID: 883) - Peru (ID: 971) - Polen (ID: 477) - Qatar (ID: 910) - Roemenië (ID: 555) - Schotland (ID: 873) - Servië (ID: 487) - Singapore (ID: 1610) - Slowakije (ID: 611) - Slovenie (ID: 605) - Tanzania (ID: 763) - Thailand (ID: 1511) - Tsjechië (ID: 489) - Tunesië (ID: 967) - Uruguay (ID: 1007) - Verenigde Arabische Emiraten (ID: 1475) - Venezuela (ID: 1546) - Vietnam (ID: 620) - Wales (ID: 1262) - Zuid-Afrika (ID: 1003) - Zuid-Korea (ID: 696) - Zweden (ID: 539) - Zwitserland (ID: 661)"""

# Extract country names and IDs using regex
matches = re.findall(r"- ([\w\s&ëéïöç-]+) \(ID: (\d+)\)", output_text)

# Convert to dictionary
football_nodes = {name.strip(): int(id_) for name, id_ in matches}

print(football_nodes)

{'Nederland': 1052, 'Engeland': 40, 'Spanje': 44, 'Italië': 43, 'Duitsland': 42, 'Frankrijk': 41, 'België': 864, 'Portugal': 551, 'Turkije': 537, 'Saudi Arabië': 987, 'VS': 595, 'Europees': 3214, 'Wereldwijd': 667, 'Americas': 9342, 'Africa': 6505, 'Andorra': 4578, 'Algerije': 1797, 'Argentinië': 1670, 'Australië': 825, 'Azerbeidzjan': 786, 'Bahrein': 1000, 'Bosnia and Herzegovina': 963, 'Brazilië': 896, 'Bulgarije': 619, 'Burundi': 519, 'Chili': 1132, 'Cambodja': 4125, 'Costa Rica': 475, 'Colombia': 1225, 'Cyprus': 1068, 'Denemarken': 479, 'Egypte': 994, 'El Salvador': 1607, 'Ethiopia': 3396, 'Finland': 692, 'Filipijnen': 5317, 'Guatemala': 1548, 'Griekenland': 513, 'Honduras': 1471, 'Hong Kong': 1353, 'Hongarije': 484, 'IJsland': 617, 'Ierland': 925, 'Indonesia': 3528, 'India': 1799, 'Israël': 542, 'Iran': 795, 'Japan': 581, 'Jordanië': 988, 'Koeweit': 1022, 'Kroatië': 544, 'Luxemburg': 1355, 'Malta': 1590, 'Maleisië': 3143, 'Mexico': 888, 'Marokko': 919, 'Nicaragua': 970, 'Noord-Ier